### **Task 2. Financial Impact of Reducing Prices**

In [2]:
# importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pycountry

sns.set(style="whitegrid")

#### Part 1. Build dataset with number of GeneXpert purchased (TGF)

Cleaning Transaction Summary Dataset:

In [23]:
# importing tgf transaction summary dataset - filtered for 'TB molecular diagnostics'
tgf_trans = pd.read_csv('/Users/henriquerodrigues/Desktop/LSE/Y2/Capstone/Data/TGF/transaction_summary.csv', encoding='utf-16', sep='\t')
# dropping first row
tgf_trans = tgf_trans.drop(index=0)

# transforming date columns into datetime format
date_columns = ['Purchase Order Date', 'Scheduled Delivery Date', 'Actual Delivery Date']

for col in date_columns:
    # issue with date columns - 'sept' to 'sep'
    tgf_trans[col] = tgf_trans[col].str.replace('Sept', 'Sep')
    tgf_trans[col] = pd.to_datetime(tgf_trans[col], format='%d-%b-%y')

# transforming columns to numerical
cols_to_num = ['Pack quantity', 'Product pack (USD)', 'Total Product Cost (USD)']

for col in cols_to_num:
    tgf_trans[col] = tgf_trans[col].str.replace(',', '').astype(float)
    tgf_trans[col] = pd.to_numeric(tgf_trans[col], errors='coerce')

# renaming columns
tgf_trans = tgf_trans.rename(columns={"Supplier/Agent/Manufacturer/Intermediatry  ": "Supplier/Agent/Manufacturer/Intermediatry",
                                      "Manufacturer  ": "Manufacturer", 
                                      "Description ": "Description",
                                      "Freight Cost ": "Freight Cost"})

# excluding regional/multicountry observations - e.g. 'Africa', 'Americas', etc.
valid_codes = [country.alpha_3 for country in pycountry.countries]
tgf_trans['Country Code'] = tgf_trans['Grant Name'].str[:3]
tgf_trans = tgf_trans[tgf_trans['Country Code'].isin(valid_codes)]

Retaining GeneXpert and calculating total purchases:

In [ ]:
# keep only Cepheid products
tgf_ceph = tgf_trans[(tgf_trans['Manufacturer']=='Cepheid') & tgf_trans['Description'].str.startswith('Cepheid')]

# creating column for number of tests in each product pack
tgf_ceph['N Units in Pack'] = tgf_ceph['Product Pack'].str.extract('(\d+)').astype(int)

# total number of tests per order
tgf_ceph['N Units Total'] = tgf_ceph['N Units in Pack'] * tgf_ceph['Pack quantity']

In [25]:
tgf_ceph.head()

,Country/Teritorry,Grant Name,Supplier/Agent/Manufacturer/Intermediatry,Manufacturer,Product Category,Product,Description,Product Pack,Purchase Order Date,Scheduled Delivery Date,...,Purchase Order Number,Invoice Currency Name,Primary Key,Status,Pack quantity,Product pack (USD),Total Product Cost (USD),Country Code,N Units in Pack,N Units Total
1,Afghanistan,AFG-T-UNDP,Direct from Mfg,Cepheid,"Diagnostic, PPE and Medical Device",TB molecular diagnostics,Cepheid GeneXpert Model GX-IV- 4 module Instru...,1 unit / test,2015-07-29,2015-10-15,...,AFG10-GP404135-1,United States Dollar,61987,Admin-Publish,4.0,17500.0,70000.0,AFG,1,4.0
2,Afghanistan,AFG-T-UNDP,Direct from Mfg,Cepheid,"Diagnostic, PPE and Medical Device",TB molecular diagnostics,Cepheid GeneXpert Model GX-IV- 4 module Instru...,1 unit / test,2017-10-05,2017-12-19,...,AFG10-GP406031-1,United States Dollar,73391,Admin-Publish,11.0,17500.0,192500.0,AFG,1,11.0
3,Afghanistan,AFG-T-UNDP,Direct from Mfg,Cepheid,"Diagnostic, PPE and Medical Device",TB molecular diagnostics,"Cepheid Xpert MTB/RIF kit (10s, 50s) [CGXMTBRIF]",50 tests,2015-07-29,2015-10-15,...,AFG10-GP404135-1,United States Dollar,61988,Admin-Publish,24.0,499.0,11976.0,AFG,50,1200.0
4,Afghanistan,AFG-T-UNDP,Direct from Mfg,Cepheid,"Diagnostic, PPE and Medical Device",TB molecular diagnostics,"Cepheid Xpert MTB/RIF kit (10s, 50s) [CGXMTBRIF]",50 tests,2015-07-29,2016-03-01,...,GP404135-1,United States Dollar,66556,Admin-Publish,95.0,499.0,47405.0,AFG,50,4750.0
5,Afghanistan,AFG-T-UNDP,Direct from Mfg,Cepheid,"Diagnostic, PPE and Medical Device",TB molecular diagnostics,"Cepheid Xpert MTB/RIF kit (10s, 50s) [CGXMTBRIF]",50 tests,2017-10-05,2017-12-19,...,AFG10-GP406031-1,United States Dollar,73392,Published,110.0,499.0,54890.0,AFG,50,5500.0


Grouping by country, year and product for final dataset:

In [26]:
tgf_final = tgf_ceph.groupby(['Country/Teritorry', 
                              tgf_ceph['Purchase Order Date'].dt.year, 
                              'Description'])['N Units Total'].sum().reset_index()

In [27]:
tgf_final['Description'].value_counts()

Description
Cepheid Xpert MTB/RIF kit (10s, 50s) [CGXMTBRIF]                                              264
Cepheid Xpert MTB/RIF Ultra (50s) [GXMTB/RIF-ULTRA-50]                                        171
Cepheid GeneXpert Model GX-IV- 4 module Instrument with desktop [GXIV-4D]                      73
Cepheid GeneXpert Model GX-IV- 4 module Instrument with laptop [GXIV-4-L]                      68
Cepheid Xpert MTB/XDR kit (10s) [GXMTB/XDR-10]                                                 57
Cepheid GeneXpert Model GX-XVI-16 module instrument with desktop[GXXVI-16-D]                   19
Cepheid GeneXpert IV system with desktop computer (two sites) [GXIV-2-D]                       10
Cepheid GeneXpert Model GX-IV R2 2-module (10-color) Instrument with laptop [GXIV-2-L-10C]     10
Cepheid GeneXpert IV system with laptop computer (two sites) [GXIV-2-L]                         9
Cepheid GeneXpert Model GX-XVI-16 module instrument with laptop [GXXVI-16-L]                    6
Cepheid 

In [28]:
# change product names
prod_dict = {"Cepheid Xpert MTB/XDR kit (10s) [GXMTB/XDR-10]": "Cepheid Xpert MTB/XDR",
   "Cepheid Xpert MTB/RIF kit (10s, 50s) [CGXMTBRIF]": "Cepheid Xpert MTB/RIF Ultra",
   "Cepheid Xpert MTB/RIF Ultra (50s) [GXMTB/RIF-ULTRA-50]": "Cepheid Xpert MTB/RIF Ultra",
   "Cepheid Xpert MTB/RIF Ultra (10s) [GXMTB/RIF-ULTRA-10]": "Cepheid Xpert MTB/RIF Ultra"}

tgf_final['Description'] = tgf_final['Description'].map(prod_dict).fillna(tgf_final['Description'])

In [29]:
tgf_final

,Country/Teritorry,Purchase Order Date,Description,N Units Total
0,Afghanistan,2015,Cepheid GeneXpert Model GX-IV- 4 module Instru...,4.0
1,Afghanistan,2015,Cepheid Xpert MTB/RIF Ultra,5950.0
2,Afghanistan,2017,Cepheid GeneXpert Model GX-IV- 4 module Instru...,11.0
3,Afghanistan,2017,Cepheid Xpert MTB/RIF Ultra,5500.0
4,Afghanistan,2018,Cepheid Xpert MTB/RIF Ultra,10000.0
...,...,...,...,...
697,Zimbabwe,2020,Cepheid Xpert MTB/RIF Ultra,229800.0
698,Zimbabwe,2021,Cepheid Xpert MTB/RIF Ultra,89400.0
699,Zimbabwe,2022,Cepheid Xpert MTB/RIF Ultra,109650.0
700,Zimbabwe,2023,Cepheid Xpert MTB/RIF Ultra,136850.0


Saving final dataset to 'Data' folder:

In [30]:
tgf_final.to_csv('/Users/henriquerodrigues/Desktop/LSE/Y2/Capstone/Data/tgf_purchases.csv')

#### Part 2. Build dataset with countries' TB funding by source and total requirements (WHO)

In [16]:
# reading in WHOs bduget dataset
fund = pd.read_csv("/Users/henriquerodrigues/Desktop/LSE/Y2/Capstone/Data/WHO/TB_budget_2024-12-05.csv")

Selecting necessary variables:
- Expected funding from domestic sources, including loans (US Dollars): `cf_tot_domestic`
- Expected funding from USAID (US Dollars): `cf_tot_usaid`
- Expected funding from the Global Fund to Fight AIDS, Tuberculosis and Malaria (US Dollars): `cf_tot_gf`
- Expected funding from other sources (US Dollars): `cf_tot_grnt`
- Total expected funding from all sources (US Dollars): `cf_tot_sources`
- Budget required for laboratory infrastructure, equipment and supplies (US Dollars): `budget_lab`
- Total budget required (US Dollars): `budget_tot`

In [17]:
fund_vars = ['country', 'year', 'cf_tot_domestic', 'cf_tot_usaid', 'cf_tot_gf', 'cf_tot_grnt', 
             'cf_tot_sources', 'budget_lab', 'budget_tot']

fund = fund[fund_vars]

# removing rows without funding data
rows_to_drop = fund[fund[fund.columns[2:]].isna().all(axis=1)]
fund = fund[~fund.index.isin(rows_to_drop.index)]

Creating 'Funding Gap' variable:

In [18]:
fund['budget_gap'] = fund['budget_tot']-fund['cf_tot_sources']

In [19]:
fund

,country,year,cf_tot_domestic,cf_tot_usaid,cf_tot_gf,cf_tot_grnt,cf_tot_sources,budget_lab,budget_tot,budget_gap
0,Afghanistan,2018,533779.0,4462530.0,3178499.0,2523675.0,10698483.0,2543262.0,10881354.0,182871.0
1,Afghanistan,2019,511854.0,2766449.0,6533095.0,3608394.0,13419792.0,2797414.0,16957452.0,3537660.0
2,Afghanistan,2020,1171864.0,3000000.0,6935551.0,6871009.0,17978424.0,3200000.0,19137128.0,1158704.0
3,Afghanistan,2021,NaN,NaN,7638546.0,2995053.0,10633599.0,1696418.0,10633599.0,0.0
4,Afghanistan,2022,269339.0,1555946.0,6876137.0,1898528.0,10599950.0,515246.0,14141821.0,3541871.0
...,...,...,...,...,...,...,...,...,...,...
1500,Zimbabwe,2020,247705.0,1870556.0,8300758.0,337233.0,10756252.0,3058011.0,31504677.0,20748425.0
1501,Zimbabwe,2021,2096683.0,3284585.0,9803368.0,911965.0,16096601.0,6529001.0,32126351.0,16029750.0
1502,Zimbabwe,2022,1000000.0,3058830.0,9487457.0,1301355.0,14847642.0,3557975.0,22656137.0,7808495.0
1503,Zimbabwe,2023,1000000.0,9378910.0,5774773.0,NaN,16153683.0,4840102.0,17359487.0,1205804.0


Saving final dataset to 'Data' folder:

In [20]:
fund.to_csv('/Users/henriquerodrigues/Desktop/LSE/Y2/Capstone/Data/funding_who.csv')

#### Part 3. Comparison with GDF Xpert orders